Clone github repo to have access to data files.

In [1]:
!git clone https://github.com/Hananxx/SentimentAnalysisPromptExp.git

Cloning into 'SentimentAnalysisPromptExp'...
remote: Enumerating objects: 29, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 29 (delta 12), reused 9 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (29/29), 43.65 KiB | 10.91 MiB/s, done.
Resolving deltas: 100% (12/12), done.


### Install and import needed packages

In [21]:
!pip install transformers torch pandas accelerate

In [31]:
import pandas as pd
import json
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import torch

### Set root path for accessing repo files

In [32]:
file_path_prefix = "SentimentAnalysisPromptExp/"

### Load dataset

In [33]:
df = pd.read_csv(file_path_prefix + 'data/app-test.csv')
print(df.head())  # Inspect the first few rows

                                            sentence  label
0                  its nice this apps is must lovely      1
1        this is really good this app is really good      1
2  ? freezes and force closes a lot on droid incr...      2
3  favorite i use this application every day is v...      1
4  ? probally the biggest flop ever. as soon as y...      2


### Load prompts

In [34]:
with open(file_path_prefix + 'prompts/zero-shot-prompt-template.json', 'r') as f:
    templates = json.load(f)
print(templates)

{'vicuna-0': "A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions.\nUSER: Please perform Sentiment Classification task. Given the sentence from {}, assign a sentiment label from ['negative', 'neutral', 'positive']. Return label only without any other text.\nASSISTANT: Sure!</s>\nUSER: Sentence: {}\nASSISTANT:", 'vicuna-jira-0': "A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions.\nUSER: Please perform Sentiment Classification task. Given the sentence from {}, assign a sentiment label from ['negative', 'positive']. Return label only without any other text.\nASSISTANT: Sure!</s>\nUSER: Sentence: {}\nASSISTANT:", 'llama2-0': "<s>[INST] <<SYS>>\nA chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's q

### Use Vicuna

In [35]:
# Load tokenizer and model
models = ['lmsys/vicuna-13b-v1.5', 'WizardLM/WizardLM-13B-V1.2', 'meta-llama/Llama-2-13b-chat-hf']
def load_model(model_name):
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # Create a text generation pipeline
    return pipeline(
        'text-generation',
        model=model_name,
        tokenizer=tokenizer,
        dtype=torch.float16,
        device_map='auto',
    )

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0


In [36]:
prompt = templates['vicuna-0']
sentences = df['sentence'].tolist() # For test purposes only the first 50 sentences.
model_output = []

for sentence in sentences:
    full_prompt = prompt.format("APP reviews", sentence)
    output = model_pipeline(
          full_prompt,
          max_length=1024,
          temperature=0.7,
          pad_token_id=tokenizer.pad_token_id,
          eos_token_id=tokenizer.eos_token_id,
          return_full_text=False
    )
    response = output[0]['generated_text'].strip()  # Extract just the label.
    # print(response)
    model_output.append(response)


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [38]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
import pandas as pd


labels = { 0: 'neutral', 1: 'positive', 2: 'negative'}
data_frame = pd.DataFrame({
    'text': df['sentence'].tolist(),
    'true_label': [labels[label_num] for label_num in df['label'].tolist()],
    'pred_label': model_output
})
accuracy = accuracy_score(data_frame['true_label'], data_frame['pred_label'])
precision, recall, f1, _ = precision_recall_fscore_support(data_frame['true_label'], data_frame['pred_label'], average='weighted')
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1:.2f}")

# print('model output', 'labelled output', sep='\t')
# for key, output in enumerate(model_output):
# print(output, labels[df['label'][key]], sep='\t')

Accuracy: 0.89
Precision: 0.91
Recall: 0.89
F1-Score: 0.90


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
